In [ ]:
# Library Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast, GradScaler
from pathlib import Path
from PIL import Image
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import torchvision.transforms.functional as TF
import random
from skimage import color

In [ ]:
# Model Architecture
class CNNEncoder(nn.Module):
    def __init__(self, in_channels=1, feature_dim=256):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, feature_dim, 3, stride=1, padding=1),
            nn.BatchNorm2d(feature_dim),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x1 = self.conv1(x)  # [B, 64, H, W]
        x2 = self.conv2(x1) # [B, 128, H/2, W/2]
        x3 = self.conv3(x2) # [B, 256, H/4, W/4]
        x4 = self.conv4(x3) # [B, 256, H/4, W/4]
        return x4, (x1, x2, x3)


class TransformerEncoder(nn.Module):
    def __init__(self, feature_dim=256, num_heads=8, num_layers=4, patch_size=4, dropout=0.1):
        super().__init__()
        self.feature_dim = feature_dim
        self.patch_size = patch_size
        
        patch_dim = patch_size * patch_size * feature_dim
        self.patch_projection = nn.Linear(patch_dim, feature_dim)
        self.pos_embed = nn.Parameter(torch.randn(1, 256, feature_dim))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=feature_dim,
            nhead=num_heads,
            dim_feedforward=feature_dim * 4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, x):
        B, C, H, W = x.shape

        # Split feature map into patches
        x = x.unfold(2, self.patch_size, self.patch_size)\
             .unfold(3, self.patch_size, self.patch_size)
        x = x.contiguous().view(B, C, -1, self.patch_size * self.patch_size)
        x = x.permute(0, 2, 1, 3).flatten(2)

        # Project patches
        x = self.patch_projection(x)
        
        # Add positional embedding
        num_patches = x.size(1)
        if num_patches <= self.pos_embed.size(1):
            pos_emb = self.pos_embed[:, :num_patches, :]
        else:
            pos_emb = F.interpolate(
                self.pos_embed.permute(0, 2, 1),
                size=num_patches,
                mode='linear',
                align_corners=False
            ).permute(0, 2, 1)
        
        x = x + pos_emb
        x = self.transformer(x)
        return x, (H, W)


class ColorDecoder(nn.Module):
    def __init__(self, feature_dim=256):
        super().__init__()
        self.up1 = nn.Sequential(
            nn.ConvTranspose2d(feature_dim, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
        
        self.up2 = nn.Sequential(
            nn.ConvTranspose2d(256 + 256, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        
        self.up3 = nn.Sequential(
            nn.Conv2d(128 + 128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        
        self.final = nn.Sequential(
            nn.Conv2d(128 + 64, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, 3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 2, 3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, x, orig_H, orig_W, skip_features=None):
        B, N, C = x.shape
        side = int(N ** 0.5)
        
        x = x.permute(0, 2, 1).view(B, C, side, side)
        x = self.up1(x)
        
        if skip_features is not None:
            skip3 = skip_features[2]
            if skip3.shape[2:] != x.shape[2:]:
                skip3 = F.interpolate(skip3, size=x.shape[2:], mode='bilinear', align_corners=False)
            x = torch.cat([x, skip3], dim=1)
        
        x = self.up2(x)
        
        if skip_features is not None:
            skip2 = skip_features[1]
            if skip2.shape[2:] != x.shape[2:]:
                skip2 = F.interpolate(skip2, size=x.shape[2:], mode='bilinear', align_corners=False)
            x = torch.cat([x, skip2], dim=1)
        
        x = self.up3(x)
        
        if skip_features is not None:
            skip1 = skip_features[0]
            if skip1.shape[2:] != x.shape[2:]:
                skip1 = F.interpolate(skip1, size=x.shape[2:], mode='bilinear', align_corners=False)
            x = torch.cat([x, skip1], dim=1)
        
        x = self.final(x)
        
        if x.shape[2] != orig_H or x.shape[3] != orig_W:
            x = F.interpolate(x, size=(orig_H, orig_W), mode="bilinear", align_corners=False)
        
        return x


class CNNTransformerColorizer(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = CNNEncoder()
        self.transformer = TransformerEncoder(dropout=0.1)
        self.decoder = ColorDecoder()

    def forward(self, x):
        orig_H, orig_W = x.shape[2], x.shape[3]
        feat, skip_features = self.encoder(x)
        trans, (enc_H, enc_W) = self.transformer(feat)
        ab = self.decoder(trans, orig_H, orig_W, skip_features)
        return ab

In [ ]:
# Loss Function
class ImprovedColorizationLoss(nn.Module):
    def __init__(self, l1_weight=1.0, perceptual_weight=0.1, saturation_weight=0.05):
        """
        Balanced loss function with all components on similar scales
        
        Arguements:
            l1_weight: Weight for L1 loss (main pixel-wise accuracy)
            perceptual_weight: Weight for MSE loss (perceptual color matching)
            saturation_weight: Weight for saturation matching
        """
        super().__init__()
        self.l1_weight = l1_weight
        self.perceptual_weight = perceptual_weight
        self.saturation_weight = saturation_weight
        self.l1_loss = nn.L1Loss()
        self.mse_loss = nn.MSELoss()
        
    def perceptual_color_loss(self, pred, target):
        # MSE loss on AB channels - much more stable than histogram matching
        return self.mse_loss(pred, target)
    
    def saturation_loss(self, pred, target):
        # Larger epsilon to prevent NaN from sqrt of negative numbers
        pred_sat = torch.sqrt(torch.clamp(pred[:, 0:1]**2 + pred[:, 1:2]**2, min=0) + 1e-6)
        target_sat = torch.sqrt(torch.clamp(target[:, 0:1]**2 + target[:, 1:2]**2, min=0) + 1e-6)
        return self.mse_loss(pred_sat, target_sat)
    
    def forward(self, pred, target):
        # L1 loss for overall color accuracy
        l1 = self.l1_loss(pred, target)
        
        # Perceptual loss (MSE) for color matching
        perceptual = self.perceptual_color_loss(pred, target)
        
        # Saturation matching
        sat = self.saturation_loss(pred, target)
        
        total_loss = (self.l1_weight * l1 + 
                     self.perceptual_weight * perceptual + 
                     self.saturation_weight * sat)
        
        return total_loss, {
            'l1': l1.item(), 
            'perceptual': perceptual.item(), 
            'saturation': sat.item()
        }

In [ ]:
# Dataloader for batch processing
class ColorizationDataset(Dataset):
    def __init__(self, data_dir, split="train", max_samples=100000, augment=True):
        data_dir = Path(data_dir)
        g_root = data_dir / "grayscale" / split
        c_root = data_dir / "color" / split

        if not g_root.exists() or not c_root.exists():
            raise RuntimeError(
                f"Expected folders:\n{g_root}\n{c_root}\nbut they were not found."
            )

        print(f"🔍 Loading {split.upper()} split")
        print(f"Searching for grayscale images in: {g_root}")

        self.gray_files = []
        for root, dirs, files in os.walk(g_root):
            jpg_files = [f for f in files if f.lower().endswith(".jpg")]
            for f in jpg_files:
                self.gray_files.append(Path(root) / f)
                if len(self.gray_files) >= max_samples:
                    break
            if len(self.gray_files) >= max_samples:
                break

        print(f"  Total grayscale images found: {len(self.gray_files)}")

        self.gray_files = sorted(self.gray_files)
        self.gray_root = g_root
        self.color_root = c_root
        self.augment = augment and (split == "train")

    def __len__(self):
        return len(self.gray_files)

    def __getitem__(self, idx):
        gray_path = self.gray_files[idx]
        rel = gray_path.relative_to(self.gray_root)
        color_path = (self.color_root / rel).with_suffix(".npy")

        # Load data
        L = np.array(Image.open(gray_path)).astype("float32")
        L = (L / 127.5) - 1.0
        AB = np.load(color_path).astype("float32")

        L = torch.tensor(L).unsqueeze(0)
        AB = torch.tensor(AB).permute(2, 0, 1)

        # Apply augmentation
        if self.augment:
            # Random horizontal flip
            if random.random() > 0.5:
                L = TF.hflip(L)
                AB = TF.hflip(AB)
            
            # Random vertical flip
            if random.random() > 0.5:
                L = TF.vflip(L)
                AB = TF.vflip(AB)
            
            # Random rotation (small angles)
            if random.random() > 0.7:
                angle = random.uniform(-15, 15)
                L = TF.rotate(L, angle)
                AB = TF.rotate(AB, angle)

        return L, AB

In [ ]:
# Training Script to fit Kaggle use case
def train_with_validation_amp(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=100, save_dir="/kaggle/working", 
                              patience=20, start_epoch=0, resume_loss_history=None):
    model.train()
    save_path = Path(save_dir)
    save_path.mkdir(exist_ok=True)
    
    # Gradient scaler for mixed precision
    scaler = GradScaler()
    
    # Initialize or resume loss history
    if resume_loss_history is not None:
        loss_history = resume_loss_history
        print(f" Resuming with {len(loss_history['train_total'])} previous epochs of history")
    else:
        loss_history = {
            'train_total': [], 'train_l1': [], 'train_perceptual': [], 'train_saturation': [],
            'val_total': [], 'val_l1': [], 'val_perceptual': [], 'val_saturation': [],
            'lr': []
        }
    
    # Find best validation loss from history
    if len(loss_history['val_total']) > 0:
        best_val_loss = min(loss_history['val_total'])
        best_epoch = loss_history['val_total'].index(best_val_loss) + 1
        print(f" Previous best validation loss: {best_val_loss:.6f} at epoch {best_epoch}")
    else:
        best_val_loss = float('inf')
    
    patience_counter = 0
    
    for epoch in range(start_epoch, epochs):
        # Training pahse
        model.train()
        train_loss = 0.0
        train_l1 = 0.0
        train_perceptual = 0.0
        train_sat = 0.0
        
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), 
                    desc=f"Epoch {epoch+1}/{epochs} [Train]")
        
        for i, (L, AB) in pbar:
            L, AB = L.to(device, non_blocking=True), AB.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            
            # Mixed precision forward pass
            with autocast():
                output = model(L)
                
                if output.shape != AB.shape:
                    output = F.interpolate(output, size=(AB.shape[2], AB.shape[3]), mode='bilinear', align_corners=False)
                
                loss, loss_components = criterion(output, AB)
            
            # Mixed precision backward pass with NaN detection
            scaler.scale(loss).backward()
            
            # Check for NaN gradients before stepping
            if torch.isnan(loss) or torch.isinf(loss):
                print(f"  NaN/Inf loss detected at batch {i}, skipping...")
                optimizer.zero_grad()
                continue
            
            scaler.unscale_(optimizer)
            
            # More aggressive gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            train_l1 += loss_components['l1']
            train_perceptual += loss_components['perceptual']
            train_sat += loss_components['saturation']
            
            pbar.set_postfix({'loss': f"{loss.item():.4f}"})
        
        avg_train_loss = train_loss / len(train_loader)
        avg_train_l1 = train_l1 / len(train_loader)
        avg_train_perceptual = train_perceptual / len(train_loader)
        avg_train_sat = train_sat / len(train_loader)
        
        # Validation Phase
        model.eval()
        val_loss = 0.0
        val_l1 = 0.0
        val_perceptual = 0.0
        val_sat = 0.0
        
        with torch.no_grad():
            for L, AB in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False):
                L, AB = L.to(device, non_blocking=True), AB.to(device, non_blocking=True)
                
                # Autocast for validation 
                with autocast():
                    output = model(L)
                    
                    if output.shape != AB.shape:
                        output = F.interpolate(output, size=(AB.shape[2], AB.shape[3]), mode='bilinear', align_corners=False)
                    
                    loss, loss_components = criterion(output, AB)
                
                val_loss += loss.item()
                val_l1 += loss_components['l1']
                val_perceptual += loss_components['perceptual']
                val_sat += loss_components['saturation']
        
        avg_val_loss = val_loss / len(val_loader)
        avg_val_l1 = val_l1 / len(val_loader)
        avg_val_perceptual = val_perceptual / len(val_loader)
        avg_val_sat = val_sat / len(val_loader)
        
        # Step scheduler
        if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
            current_lr = optimizer.param_groups[0]['lr']
        elif scheduler is not None:
            scheduler.step()
            current_lr = scheduler.get_last_lr()[0]
        else:
            current_lr = optimizer.param_groups[0]['lr']
        
        # Store history
        loss_history['train_total'].append(avg_train_loss)
        loss_history['train_l1'].append(avg_train_l1)
        loss_history['train_perceptual'].append(avg_train_perceptual)
        loss_history['train_saturation'].append(avg_train_sat)
        loss_history['val_total'].append(avg_val_loss)
        loss_history['val_l1'].append(avg_val_l1)
        loss_history['val_perceptual'].append(avg_val_perceptual)
        loss_history['val_saturation'].append(avg_val_sat)
        loss_history['lr'].append(current_lr)
        
        print(f"\nEpoch [{epoch+1}/{epochs}]")
        print(f"Train Loss: {avg_train_loss:.6f} (L1: {avg_train_l1:.6f}, Perceptual: {avg_train_perceptual:.6f}, Sat: {avg_train_sat:.6f})")
        print(f"Val Loss:   {avg_val_loss:.6f} (L1: {avg_val_l1:.6f}, Perceptual: {avg_val_perceptual:.6f}, Sat: {avg_val_sat:.6f})")
        print(f"LR: {current_lr:.2e}")
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            
            if hasattr(model, 'module'):
                model_to_save = model.module
            else:
                model_to_save = model
            
            best_model_path = save_path / "best_model.pth"
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model_to_save.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': best_val_loss,
                'loss_history': loss_history,
            }, best_model_path)
            print(f" New best model saved;; Val loss: {best_val_loss:.6f}")
        else:
            patience_counter += 1
            print(f"  No improvement for {patience_counter}/{patience} epochs")
            
            if patience_counter >= patience:
                print(f"\n  Early stopping triggered after {epoch+1} epochs")
                print(f"  Best validation loss: {best_val_loss:.6f}")
                break
        
        # Create checkpoints incase of deathly crashes
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f"\n  Checkpoint at epoch {epoch+1}")
            
            if hasattr(model, 'module'):
                model_to_save = model.module
            else:
                model_to_save = model
            
            checkpoint_path = save_path / f"checkpoint_epoch_{epoch+1:03d}.pth"
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model_to_save.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict() if hasattr(scheduler, 'state_dict') else None,
                'loss_history': loss_history,
                'val_loss': avg_val_loss,
            }, checkpoint_path)
            print(f"  Checkpoint saved: {checkpoint_path}")
            
            try:
                visualization_path = save_path / f"visualization_epoch_{epoch+1:03d}.png"
                generate_visualization(model_to_save, val_loader.dataset, device, save_path=visualization_path, epoch=epoch+1, saturation_boost=1.6)
                print(f"  Visualization saved")
            except Exception as e:
                print(f"  Visualization failed: {e}")
            
            plot_loss_history_with_validation(loss_history, save_path / f"loss_plot_epoch_{epoch+1:03d}.png")
            print(f"  Loss plot saved")
        
        # Save checkpoint every 5 epochs incase of death ly crash
        elif (epoch + 1) % 5 == 0:
            if hasattr(model, 'module'):
                model_to_save = model.module
            else:
                model_to_save = model
            
            checkpoint_path = save_path / f"checkpoint_epoch_{epoch+1:03d}.pth"
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model_to_save.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict() if hasattr(scheduler, 'state_dict') else None,
                'loss_history': loss_history,
                'val_loss': avg_val_loss,
            }, checkpoint_path)
            print(f" Quick checkpoint saved: {checkpoint_path}")
    
    print("\n If you see this, it has been more than 12 hours")
    return model, loss_history

In [ ]:
# Visualisation
def generate_visualization(model, dataset, device, num_samples=3, saturation_boost=1.5, save_path=None, epoch=None):
    # Visualize model predictions with higher saturation boost

    model.eval()
    idxs = np.random.choice(len(dataset), min(num_samples, len(dataset)), replace=False)
    
    fig = plt.figure(figsize=(15, 4 * len(idxs)))
    
    if epoch is not None:
        plt.suptitle(f"Epoch {epoch} - Model Predictions", fontsize=16, y=0.995)
    
    for i, idx in enumerate(idxs):
        L, AB = dataset[idx]
        
        # Get prediction
        with torch.no_grad():
            pred_AB = model(L.unsqueeze(0).to(device)).cpu().squeeze(0)
        
        # Apply saturation boost
        pred_AB_boosted = pred_AB * saturation_boost
        pred_AB_boosted = torch.clamp(pred_AB_boosted, -1.0, 1.0)
        
        # Convert to numpy
        L_np = L.squeeze().numpy()
        pred_AB_np = pred_AB.permute(1, 2, 0).numpy()
        pred_AB_boosted_np = pred_AB_boosted.permute(1, 2, 0).numpy()
        AB_gt_np = AB.permute(1, 2, 0).numpy()
        
        # Convert back to CIELAB range
        L_lab = (L_np + 1.0) * 50.0
        pred_AB_lab = pred_AB_np * 128.0
        pred_AB_boosted_lab = pred_AB_boosted_np * 128.0
        AB_gt_lab = AB_gt_np * 128.0
        
        # Reconstruct LAB images
        pred_lab = np.zeros((L_lab.shape[0], L_lab.shape[1], 3))
        pred_lab[:, :, 0] = L_lab
        pred_lab[:, :, 1:] = pred_AB_lab
        
        pred_boosted_lab = np.zeros_like(pred_lab)
        pred_boosted_lab[:, :, 0] = L_lab
        pred_boosted_lab[:, :, 1:] = pred_AB_boosted_lab
        
        gt_lab = np.zeros_like(pred_lab)
        gt_lab[:, :, 0] = L_lab
        gt_lab[:, :, 1:] = AB_gt_lab
        
        # Convert LAB to RGB
        pred_rgb = color.lab2rgb(pred_lab)
        pred_boosted_rgb = color.lab2rgb(pred_boosted_lab)
        gt_rgb = color.lab2rgb(gt_lab)
        
        # Plot
        plt.subplot(len(idxs), 4, 4 * i + 1)
        plt.imshow(L_np, cmap="gray", vmin=-1, vmax=1)
        plt.title("Input Grayscale")
        plt.axis("off")

        plt.subplot(len(idxs), 4, 4 * i + 2)
        plt.imshow(np.clip(pred_rgb, 0, 1))
        plt.title("Predicted Colour")
        plt.axis("off")
        
        plt.subplot(len(idxs), 4, 4 * i + 3)
        plt.imshow(np.clip(pred_boosted_rgb, 0, 1))
        plt.title(f"Boosted (x{saturation_boost})")
        plt.axis("off")

        plt.subplot(len(idxs), 4, 4 * i + 4)
        plt.imshow(np.clip(gt_rgb, 0, 1))
        plt.title("Ground Truth")
        plt.axis("off")
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()


def plot_loss_history_with_validation(loss_history, save_path=None):
    #Plot training and validation losses
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    epochs = range(1, len(loss_history['train_total']) + 1)
    
    # Total loss
    ax1.plot(epochs, loss_history['train_total'], label='Train', linewidth=2)
    ax1.plot(epochs, loss_history['val_total'], label='Validation', linewidth=2)
    ax1.set_title('Total Loss', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # L1 loss
    ax2.plot(epochs, loss_history['train_l1'], label='Train', linewidth=2)
    ax2.plot(epochs, loss_history['val_l1'], label='Validation', linewidth=2)
    ax2.set_title('L1 Loss', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('L1 Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Perceptual loss
    ax3.plot(epochs, loss_history['train_perceptual'], label='Train', linewidth=2)
    ax3.plot(epochs, loss_history['val_perceptual'], label='Validation', linewidth=2)
    ax3.set_title('Perceptual Loss (MSE)', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Perceptual Loss')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Learning rate
    ax4.plot(epochs, loss_history['lr'], linewidth=2, color='green')
    ax4.set_title('Learning Rate', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('Learning Rate')
    ax4.set_yscale('log')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()

In [ ]:
# Resume Training due to 12hr limit
def find_checkpoint_in_input():
    input_dir = Path("/kaggle/input")
    
    if not input_dir.exists():
        print("No /kaggle/input directory found")
        return None
    
    checkpoint_files = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.pth'):
                full_path = os.path.join(root, file)
                checkpoint_files.append(full_path)
                print(f"  Found: {full_path}")
    
    if len(checkpoint_files) == 0:
        print("  No .pth files found")
        return None
    elif len(checkpoint_files) == 1:
        print(f"\n Using checkpoint: {checkpoint_files[0]}")
        return checkpoint_files[0]
    else:
        print(f"\n  Found {len(checkpoint_files)} checkpoint files. Which is: {checkpoint_files[0]}")
        print("   If you want a different one, specify it manually in main(resume_from=...) !!!")
        return checkpoint_files[0]


def load_checkpoint(checkpoint_path, model, optimizer=None, scheduler=None):
    """
    Load model checkpoint and optionally resume training state
    
    Args:
        checkpoint_path: Path to .pth checkpoint file
        model: Model to load weights into
        optimizer: Optional optimizer to restore state
        scheduler: Optional scheduler to restore state
    
    Returns:
        start_epoch: Epoch to resume from
        loss_history: Previous training history
    """
    print(f"  Loading checkpoint from: {checkpoint_path}")
    
    # Check if file exists
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint not found at: {checkpoint_path}")
    
    checkpoint = torch.load(checkpoint_path)
    
    # Load model weights with compatibility for DataParallel
    state_dict = checkpoint['model_state_dict']
    
    # Handle DataParallel compatibility
    # If saved with DataParallel but loading without, or vice versa
    try:
        if hasattr(model, 'module'):
            # Current model has DataParallel wrapper
            try:
                model.module.load_state_dict(state_dict)
            except:
                # Checkpoint might not have 'module.' prefix, add it
                new_state_dict = {'module.' + k: v for k, v in state_dict.items()}
                model.load_state_dict(new_state_dict)
        else:
            # Current model doesn't have DataParallel wrapper
            try:
                model.load_state_dict(state_dict)
            except:
                # Checkpoint has 'module.' prefix, remove it
                new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
                model.load_state_dict(new_state_dict)
        print(f"  Model weights loaded from epoch {checkpoint['epoch']}")
    except Exception as e:
        print(f"  Error loading model weights: {e}")
        print("Trying alternative loading method...")
        # Last resort - try both ways
        if hasattr(model, 'module'):
            new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
            model.module.load_state_dict(new_state_dict)
        else:
            new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
            model.load_state_dict(new_state_dict)
        print(f"  Model weights loaded successfully using fallback method")
    
    # Load optimizer state
    if optimizer is not None and 'optimizer_state_dict' in checkpoint:
        try:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            print(f"  Optimizer state restored")
        except Exception as e:
            print(f"  Warning: Could not restore optimizer state: {e}")
            print("   Continuing with fresh optimizer state...")
    
    # Load scheduler state
    if scheduler is not None and 'scheduler_state_dict' in checkpoint and checkpoint['scheduler_state_dict'] is not None:
        try:
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            print(f"  Scheduler state restored")
        except Exception as e:
            print(f"  Warning: Could not restore scheduler state: {e}")
            print("   Continuing with fresh scheduler state...")
    
    # Get training info
    start_epoch = checkpoint.get('epoch', 0)
    loss_history = checkpoint.get('loss_history', None)
    
    print(f"  Resuming from epoch {start_epoch}")
    if 'val_loss' in checkpoint:
        print(f"  Previous validation loss at this checkpoint: {checkpoint['val_loss']:.6f}")
    
    return start_epoch, loss_history

In [ ]:
# Dataset Loader
def find_dataset_path():
    """Automatically find the dataset path in Kaggle"""
    input_dir = Path("/kaggle/input/150k-data")
    
    if not input_dir.exists():
        print("/kaggle/input/150k-data directory not found")
        return None
    
    available_datasets = [d for d in input_dir.iterdir() if d.is_dir()]
    print(f"Available datasets: {[d.name for d in available_datasets]}")
    
    # Look for processed data
    for dataset in available_datasets:
        potential_path = dataset / "processed"
        if potential_path.exists():
            print(f"Found processed data at: {potential_path}")
            return str(potential_path)
    
    # Look for train/grayscale structure directly
    for dataset in available_datasets:
        potential_train = dataset / "grayscale" / "train"
        if potential_train.exists():
            print(f"Found data at: {dataset}")
            return str(dataset)
    
    # Use the first dataset as fallback
    if available_datasets:
        fallback = available_datasets[0]
        print(f"Using fallback dataset: {fallback}")
        return str(fallback)
    
    print("No datasets found")
    return None

In [ ]:
# Main Training Loop
def main(resume_from="auto"):
    """
    Main training function
    
    Args:
        resume_from: Path to checkpoint to resume from
                     - "auto": Automatically search in /kaggle/input
                     - None or False: Start fresh training
                     - "/path/to/checkpoint.pth": Load specific checkpoint
    """
    # Auto-detect checkpoint if requested
    if resume_from == "auto":
        resume_from = find_checkpoint_in_input()
        if resume_from is None:
            print("  No checkpoint found, starting fresh training, let wait!!!")
    elif resume_from == False or resume_from == "":
        resume_from = None
    
    # GPU configuration
    num_gpus = torch.cuda.device_count()
    print(f" Found {num_gpus} GPU(s)")
    
    for i in range(num_gpus):
        gpu_props = torch.cuda.get_device_properties(i)
        print(f"  - GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"    Memory: {gpu_props.total_memory / 1024**3:.1f} GB")
    
    # Setup device and model
    if num_gpus > 1:
        device = torch.device("cuda:0")
        print(f"Using {num_gpus} GPUs with DataParallel")
        model = CNNTransformerColorizer()
        model = DataParallel(model)
        model = model.to(device)
        batch_size = 8 * num_gpus
    else:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = CNNTransformerColorizer().to(device)
        batch_size = 8
        print(f"Using device: {device}")

    # Auto-detect dataset path
    data_dir = find_dataset_path()
    if data_dir is None:
        print(" No dataset found :(")
        return
    
    print(f"\n Using data directory: {data_dir}\n")
    
    # Load datasets with augmentation
    try:
        # Set ur training data and validation data here
        train_dataset = ColorizationDataset(data_dir, split="train", max_samples=200000, augment=True)
        val_dataset = ColorizationDataset(data_dir, split="val", max_samples=4000, augment=False)
        
        train_loader = DataLoader(
            train_dataset, 
            batch_size=batch_size, 
            shuffle=True, 
            num_workers=2,
            pin_memory=True
        )
        
        val_loader = DataLoader(
            val_dataset, 
            batch_size=batch_size, 
            shuffle=False, 
            num_workers=2,
            pin_memory=True
        )
        
        print(f" Training with {len(train_dataset)} images")
        print(f" Validating with {len(val_dataset)} images")
        print(f" Batch size: {batch_size}")
        
    except Exception as e:
        print(f" Error loading dataset: {e}")
        print("\nAvailable directories:")
        for root, dirs, files in os.walk(data_dir):
            level = root.replace(str(data_dir), '').count(os.sep)
            if level < 3:  # Limit depth
                indent = ' ' * 2 * level
                print(f'{indent}{os.path.basename(root)}/')
        return

    # Setup training components
    print("\n  Configuring training...")
    
    # Criterion
    criterion = ImprovedColorizationLoss(
        l1_weight=0.8,
        perceptual_weight=0.1,
        saturation_weight=0.15 
    )
    
    # Adam Optimizer
    optimizer = torch.optim.AdamW(
        model.parameters(), 
        lr=8e-4,  
        weight_decay=1e-4,
        eps=1e-7
    )
    
    # Learning rate scheduler 
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='min', 
        factor=0.5,
        patience=12,  
        verbose=True, 
        min_lr=1e-6
    )
    
    print(" Training starting soon....")
    start_epoch = 0
    resume_loss_history = None
    
    if resume_from is not None:
        print(" RESUMING FROM CHECKPOINT")
        start_epoch, resume_loss_history = load_checkpoint(
            resume_from, model, optimizer, scheduler
        )
    
    if resume_from:
        print(f" RESUMING TRAINING FROM EPOCH {start_epoch}")
        print(" Using new weights")
    else:
        print(" Starting new model from scratch")

    
    # Final Training Parameters
    model, loss_history = train_with_validation_amp(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        epochs=150,
        save_dir="/kaggle/working",
        patience=25,
        start_epoch=start_epoch,
        resume_loss_history=resume_loss_history
    )

    print("\n")
    print(" Congrats on waiting, training is completed")
    print(f" All outputs saved to: /kaggle/working/")
    print("\nSaved files:")
    print("  - best_model.pth - Best model based on validation loss")
    print("  - checkpoint_epoch_XXX.pth - Periodic checkpoints")
    print("  - visualization_epoch_XXX.png - Sample predictions")
    print("  - loss_plot_epoch_XXX.png - Training curves")
    
    # Generate final visualization on best model
    print("\n Generating final visualization from best model...")
    try:
        best_checkpoint = torch.load("/kaggle/working/best_model.pth")
        if hasattr(model, 'module'):
            model.module.load_state_dict(best_checkpoint['model_state_dict'])
        else:
            model.load_state_dict(best_checkpoint['model_state_dict'])
        
        generate_visualization(
            model.module if hasattr(model, 'module') else model,
            val_dataset, 
            device, 
            num_samples=5,
            saturation_boost=1.6,
            save_path="/kaggle/working/final_visualization.png"
        )
        print(" Final visualization saved: final_visualization.png")
        
        # Generate one with higher saturation for comparison
        generate_visualization(
            model.module if hasattr(model, 'module') else model,
            val_dataset, 
            device, 
            num_samples=5,
            saturation_boost=2.2, 
            save_path="/kaggle/working/final_visualization_boosted.png"
        )
        print(" High saturation visualization saved: final_visualization_boosted.png")
        
    except Exception as e:
        print(f" Error: {e}")
    
    # Print final statistics
    print("\n Training Statistics:")
    print(f"   Total epochs trained: {len(loss_history['train_total'])}")
    print(f"   Best validation loss: {min(loss_history['val_total']):.6f}")
    print(f"   Final learning rate: {loss_history['lr'][-1]:.2e}")
    
    print("\n Check ur working directory at the side bar. Click refresh if it does not show up")

In [ ]:
# Run 
if __name__ == "__main__":
    # Start new training
    main(resume_from=None)
    
    # Resume from checkpoint (copy path and paste it here)
    # main(resume_from="/kaggle/input/your-checkpoint/checkpoint.pth")
    